In [381]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer


# Dealing with missing data

In this notebook we will compare different ways to deal with missing data in a data set. Firstly we will import the full data into a dataframe called `X`. We will also import additional data we will use to test the model into `X_test`.

In [382]:
X = pd.read_csv('./data/train.csv', index_col='Id')
X_test = pd.read_csv('./data/test.csv', index_col='Id')

We can run a quick inspection of the data we are dealing with using the `shape` property and `head()` method.

In [383]:
print(X.shape)
print(X.head())

(1460, 80)
    MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                    
1           60       RL         65.0     8450   Pave   NaN      Reg   
2           20       RL         80.0     9600   Pave   NaN      Reg   
3           60       RL         68.0    11250   Pave   NaN      IR1   
4           70       RL         60.0     9550   Pave   NaN      IR1   
5           60       RL         84.0    14260   Pave   NaN      IR1   

   LandContour Utilities LotConfig  ... PoolArea PoolQC Fence MiscFeature  \
Id                                  ...                                     
1          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
2          Lvl    AllPub       FR2  ...        0    NaN   NaN         NaN   
3          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
4          Lvl    AllPub    Corner  ...        0    NaN   NaN         NaN   
5          Lvl    AllPub     

In [384]:
print(X_test.shape)
print(X_test.head())

(1459, 79)
      MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                      
1461          20       RH         80.0    11622   Pave   NaN      Reg   
1462          20       RL         81.0    14267   Pave   NaN      IR1   
1463          60       RL         74.0    13830   Pave   NaN      IR1   
1464          60       RL         78.0     9978   Pave   NaN      IR1   
1465         120       RL         43.0     5005   Pave   NaN      IR1   

     LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC  Fence  \
Id                                    ...                                      
1461         Lvl    AllPub    Inside  ...         120        0    NaN  MnPrv   
1462         Lvl    AllPub    Corner  ...           0        0    NaN    NaN   
1463         Lvl    AllPub    Inside  ...           0        0    NaN  MnPrv   
1464         Lvl    AllPub    Inside  ...           0        0    NaN    NaN 

Firstly, in this model we will only deal with numerical data, so we want to remove all columns that don't have numbers using `select_dtypes()`

In [385]:
X = X.select_dtypes(exclude='object')
X_test = X_test.select_dtypes(exclude='object')

In [386]:
print(X.shape)
print(X.head())

(1460, 37)
    MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                          
1           60         65.0     8450            7            5       2003   
2           20         80.0     9600            6            8       1976   
3           60         68.0    11250            7            5       2001   
4           70         60.0     9550            7            5       1915   
5           60         84.0    14260            8            5       2000   

    YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  ...  WoodDeckSF  \
Id                                                    ...               
1           2003       196.0         706           0  ...           0   
2           1976         0.0         978           0  ...         298   
3           2002       162.0         486           0  ...           0   
4           1970         0.0         216           0  ...           0   
5          

In [387]:
print(X_test.shape)
print(X_test.head())

(1459, 36)
      MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                            
1461          20         80.0    11622            5            6       1961   
1462          20         81.0    14267            6            6       1958   
1463          60         74.0    13830            5            5       1997   
1464          60         78.0     9978            6            6       1998   
1465         120         43.0     5005            8            5       1992   

      YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  ...  GarageArea  \
Id                                                      ...               
1461          1961         0.0       468.0       144.0  ...       730.0   
1462          1958       108.0       923.0         0.0  ...       312.0   
1463          1998         0.0       791.0         0.0  ...       482.0   
1464          1998        20.0       602.0         0.0  ... 

We cleared the data so we only using what we want to use. Now we can check if `X` and `X_test` has any `NaN` values.

In [388]:
X_nan_columns=X.isna().sum()[X.isna().sum()>0]
X_nan_columns

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [389]:
X_test_nan_columns=X_test.isna().sum()[X_test.isna().sum()>0]
X_test_nan_columns

LotFrontage     227
MasVnrArea       15
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt      78
GarageCars        1
GarageArea        1
dtype: int64

Observing the missing variable from `X` and `X_test` we can see that `LotFrontage`, `MasVnrArea` and `GarageYrBlt` are the features with the most `NaN` values. 

Our target variable is `SalesPrice`. Since this column is not present in either tests earlier we can proceed as normal. If `NaN` values were present in the target variable we would need to removed the whole row.

Now, we want to assign the target variable to `y` and remove it from `X`.

In [390]:
y = X['SalePrice'] # assign target variable to `y`
X.drop('SalePrice', axis ='columns', inplace= True) # remove column SalePrice from `X`

We have the following variables so far:
* `X` and `y`: These data will be used to create the final model once we find it.
* `X_test` and `y_test`: These data will be used to test the final model.

We will now create the following variables using `X` and `y` and the `train_test_split()` method:
* `X_train` and `y_train`: This data will be used to create several models, the data is essentially the control variable in a experiment to find the best model.
* `X_val` and `y_val`: This data will be used to validate the models we create to determine which is the best one.

In [391]:
X_train, X_val, y_train, y_val = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

Now we can create a function to check the MAE of each model we try.

In [392]:
def mae_score(X_train, X_val, y_train, y_val):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train,y_train) # fit model with data
    predictions = model.predict(X_val) # create predictions
    return mean_absolute_error(y_val, predictions) # estimate mae of predictions based on the known values

# Using amputation as a method to deal with missing data

By completely removing columns that have `NaN` values we can improve the model. However this usually only works when a large number of the values in the column are missing. By removing a whole column we would be removing potentially useful data.

The first thing we do is to find the columns that have `NaN` values. We can obtain this using the `index` property of `X_nan_columns`. We can pass the columns to `drop()` method to drope them.

In [393]:
reduced_X_train = X_train.drop(X_nan_columns.index, axis=1)
reduced_X_valid = X_val.drop(X_nan_columns.index, axis=1)

Now we can use `mae_score()`. We will compare the MAE to another model where we used a different method to deal with the missing data.

In [394]:
print(mae_score(reduced_X_train, reduced_X_valid, y_train, y_val))

17837.82570776256


# Dealing with data using imputation method

Imputation relates to substituting the `NaN` values with another value. For example, we can calculate the mean value in a column and use it to replace the `NaN`.

In [395]:
imputer = SimpleImputer(strategy='mean')
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_val))

The imputing process removed the column names.

In [396]:
imputed_X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,20.0,90.000000,11694.0,9.0,5.0,2007.0,2007.0,452.0,48.0,0.0,...,774.0,0.0,108.0,0.0,0.0,260.0,0.0,0.0,7.0,2007.0
1,20.0,60.000000,6600.0,5.0,5.0,1962.0,1962.0,0.0,0.0,0.0,...,308.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2009.0
2,30.0,80.000000,13360.0,5.0,7.0,1921.0,2006.0,0.0,713.0,0.0,...,432.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,8.0,2009.0
3,20.0,69.614017,13265.0,8.0,5.0,2002.0,2002.0,148.0,1218.0,0.0,...,857.0,150.0,59.0,0.0,0.0,0.0,0.0,0.0,7.0,2008.0
4,20.0,118.000000,13704.0,7.0,5.0,2001.0,2002.0,150.0,0.0,0.0,...,843.0,468.0,81.0,0.0,0.0,0.0,0.0,0.0,1.0,2006.0


In [397]:
imputed_X_valid.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,20.0,69.614017,32668.0,6.0,3.0,1957.0,1975.0,103.481067,1219.0,0.0,...,484.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,3.0,2007.0
1,50.0,79.000000,9490.0,6.0,7.0,1941.0,1950.0,0.000000,403.0,165.0,...,240.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,8.0,2006.0
2,50.0,69.614017,7015.0,5.0,4.0,1950.0,1950.0,161.000000,185.0,0.0,...,352.0,0.0,0.0,248.0,0.0,0.0,0.0,0.0,7.0,2009.0
3,60.0,83.000000,10005.0,7.0,5.0,1977.0,1977.0,299.000000,392.0,0.0,...,505.0,288.0,117.0,0.0,0.0,0.0,0.0,0.0,3.0,2008.0
4,160.0,21.000000,1680.0,6.0,5.0,1971.0,1971.0,381.000000,0.0,0.0,...,264.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0


We can put the columns back very easily.

In [398]:
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_val.columns

In [399]:
imputed_X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20.0,90.000000,11694.0,9.0,5.0,2007.0,2007.0,452.0,48.0,0.0,...,774.0,0.0,108.0,0.0,0.0,260.0,0.0,0.0,7.0,2007.0
1,20.0,60.000000,6600.0,5.0,5.0,1962.0,1962.0,0.0,0.0,0.0,...,308.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2009.0
2,30.0,80.000000,13360.0,5.0,7.0,1921.0,2006.0,0.0,713.0,0.0,...,432.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,8.0,2009.0
3,20.0,69.614017,13265.0,8.0,5.0,2002.0,2002.0,148.0,1218.0,0.0,...,857.0,150.0,59.0,0.0,0.0,0.0,0.0,0.0,7.0,2008.0
4,20.0,118.000000,13704.0,7.0,5.0,2001.0,2002.0,150.0,0.0,0.0,...,843.0,468.0,81.0,0.0,0.0,0.0,0.0,0.0,1.0,2006.0


Now, we have the imputed data, we can use `mae_score()` and compare the results.

In [400]:
print(mae_score(imputed_X_train,imputed_X_valid, y_train, y_val))

18062.894611872147


Imputation performed slightly worse! But why?

Sometimes, missing data can tell you some information about the data. For example, the feature `GarageYrBuilt` tells you what year a garage was built, but many houses do not have a garage, so for many houses this feature will have a `NaN` value.

With this information we can start to think what we could do next, the know that droping the column performs better, but we are sill droping the column and therefore losing data. We could replace the `NaN` values with 0. This means that we take into count that a house does not have a garage.

In [401]:
X_train_nan_0 = X_train.copy()
X_train_nan_0['GarageYrBlt'].fillna(value=0, inplace=True)
X_train_nan_0 = pd.DataFrame(imputer.transform(X_train_nan_0))
X_train_nan_0.columns = X_train.columns

X_val_nan_0 = X_val.copy()
X_val_nan_0['GarageYrBlt'].fillna(value=0, inplace=True)
X_val_nan_0 = pd.DataFrame(imputer.transform(X_val_nan_0))
X_val_nan_0.columns = X_val.columns

print(mae_score(X_train_nan_0,X_val_nan_0, y_train, y_val))

18053.780251141554


This new imputation model performed better `mae = 18053` than the first one `mae = 18062` but not by much. Column drop still performed better `mae = 17837`. We are probably right to fill the `NaN` values of the `GarageYrBlt` with `0`. We can now focus on another column, `LotFrontage`.

After some research, Lot frontage is the lot area in front of the house. Not all houses have a front lot. This means that the `NaN` values may be houses that don't have a front lot area. We can try to fill the `NaN` values with `0` to see if there is a improvement in the model.

In [402]:
X_train_fillnan = X_train.copy()
X_train_fillnan['GarageYrBlt'].fillna(value=0, inplace=True)
X_train_fillnan['LotFrontage'].fillna(value=0, inplace=True)
X_train_fillnan = pd.DataFrame(imputer.transform(X_train_fillnan))
X_train_fillnan.columns = X_train.columns

X_val_fillnan = X_val.copy()
X_val_fillnan['GarageYrBlt'].fillna(value=0, inplace=True)
X_val_fillnan['LotFrontage'].fillna(value=0, inplace=True)
X_val_fillnan = pd.DataFrame(imputer.transform(X_val_fillnan))
X_val_fillnan.columns = X_val.columns

print(mae_score(X_train_fillnan,X_val_fillnan, y_train, y_val))


18050.753778538812


A very small improvement, in the mae. Now, in this model we are using 36 featured, the amputation method uses 33. This may be a overfitting problem. But to make sure we will also fill NaN of `MasVnrArea`.

In [403]:
X_train_fillnan_1= X_train.copy()
X_train_fillnan_1['GarageYrBlt'].fillna(value=0, inplace=True)
X_train_fillnan_1['LotFrontage'].fillna(value=0, inplace=True)
X_train_fillnan_1['MasVnrArea'].fillna(value=0, inplace=True)
#X_train_fillnan_1 = pd.DataFrame(imputer.transform(X_train_fillnan_1))
#X_train_fillnan_1.columns = X_train.columns

X_val_fillnan_1 = X_val.copy()
X_val_fillnan_1['GarageYrBlt'].fillna(value=0, inplace=True)
X_val_fillnan_1['LotFrontage'].fillna(value=0, inplace=True)
X_val_fillnan_1['MasVnrArea'].fillna(value=0, inplace=True)
#X_val_fillnan_1 = pd.DataFrame(imputer.transform(X_val_fillnan_1))
#X_val_fillnan_1.columns = X_val.columns

print(mae_score(X_train_fillnan_1,X_val_fillnan_1, y_train, y_val))


18017.665970319635


That was a fairly bigger improvement, `18017.665970319635`. However amputation still performes better.

We can try ising a different strategy for the imputer, we can use the `median` strategy. We will need to create a new imputer for this.

In [405]:
median_imputer = SimpleImputer(strategy='median')
median_imputer.fit(X_train)

SimpleImputer(strategy='median')

In [411]:
X_train_median_1 = pd.DataFrame(median_imputer.transform(X_train))
X_valid_median_1 = pd.DataFrame(median_imputer.transform(X_val))
X_train_median_1.columns = X_train.columns
X_valid_median_1.columns = X_val.columns

print(mae_score(X_train_median_1,X_valid_median_1, y_train, y_val))

17791.59899543379


In [430]:
X_train_median_2 = X_train.copy()
X_train_median_2['MasVnrArea'].fillna(value=0, inplace=True)
#X_train_median_2['GarageYrBlt'].fillna(value=0, inplace=True)
X_train_median_2['LotFrontage'].fillna(value=0, inplace=True)
X_train_median_2 = pd.DataFrame(median_imputer.transform(X_train_median_2))
X_train_median_2.columns = X_train.columns

X_valid_median_2 = X_val.copy()
X_valid_median_2['MasVnrArea'].fillna(value=0, inplace=True)
#X_valid_median_2['GarageYrBlt'].fillna(value=0, inplace=True)
X_valid_median_2['LotFrontage'].fillna(value=0, inplace=True)
X_valid_median_2 = pd.DataFrame(median_imputer.transform(X_valid_median_2))
X_valid_median_2.columns = X_val.columns
X_valid_median_2.isna().any().sum()

print(mae_score(X_train_median_2,X_valid_median_2, y_train, y_val))

18128.13852739726


* Amputation:                             17837.82570776256
* **median method:                          17791.59899543379**
* **MasVnrArea:                             17791.59899543379**
* MasVnrArea, GarageYrBlt:                17807.438333333328
* MasVnrArea, GarageYrBlt, LotFrontage:   18017.665970319635
* LotFrontage:                            18128.13852739726
* LotFrontage, GarageYrBlt:               18017.665970319635
* GarageYrBlt:                            17807.438333333328
* MasVnrArea, LotFrontage:                18128.13852739726


Now we know that preprocessing method we want to use. We will use the imputing with `median` strategy. Now that we know this is the best approach we can start to preprocess the `X` and `X_test` data.

In [442]:
final_imputer = SimpleImputer(strategy='median')
final_imputer.fit(X)
final_X_test= pd.DataFrame(final_imputer.transform(X_test), columns=X_test.columns)
final_X = pd.DataFrame(final_imputer.transform(X), columns=X.columns)

In [454]:
X.isna().any().sum()

3

Now, we will create a model and fit the preprocessed data into it.

In [450]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X,y)
predictions = model.predict(final_X_test)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values